In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
import cv2
import pandas as pd
%env SM_FRAMEWORK=tf.keras
import segmentation_models as sm
import tensorflow as tf
# import keras.callbacks
# from keras import backend as K
import tensorflow.keras
from tensorflow.keras import backend as K
import albumentations
import random
import gc
from sklearn.model_selection import train_test_split
import PIL.Image as Image  



# os.environ['KERAS_BACKEND']='tensorflow'
os.environ["CUDA_VISIBLE_DEVICES"]="0"

seed= 1
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

# Function set


In [ ]:
def minmax_normal(img):
    img = (img-0)/255  
    return img

In [ ]:
def read_data(img_path,label_path):
    img_name = os.listdir(img_path)
    label_name = os.listdir(label_path)
    
    img_all = []
    label_all = []
    img_name_all = []
    
    for img_tmp in img_name:
        img = cv2.imread(img_path + '/' + img_tmp)
        label = cv2.imread(label_path + '/' + img_tmp,0)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
#         #resize
#         resize_number = 256
#         img = cv2.resize(img,(resize_number,resize_number))
#         label = cv2.resize(label,(resize_number,resize_number))
# #         img = minmax_normal(img)
# #         label = minmax_normal(label)
        label = label.reshape(label.shape[0],label.shape[1],1)
        img_all.append(img)
        label_all.append(label)
        img_name_all.append(img_tmp)
        
    return np.array(img_all),np.array(label_all),img_name_all

In [ ]:
def data_aug(train_x,train_y):

    new_x = []
    new_y = []
    
    transform = albumentations.Compose([
        albumentations.VerticalFlip(p=0.5),
        albumentations.HorizontalFlip(p=0.5),
#         albumentations.ChannelShuffle(p=0.5),
        albumentations.Blur(blur_limit=3, p=0.5),
        albumentations.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
        albumentations.ElasticTransform(alpha_affine=15, border_mode=1, p=0.5),
        albumentations.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=20,
                                        interpolation=cv2.INTER_LINEAR, border_mode=cv2.BORDER_REPLICATE, p=0.5)
    ])
    train_x = train_x.astype(np.float32)
    train_y = train_y.astype(np.float32)
    for i in range(len(train_x)):
        
        #new_x.append(train_x[i])
        #new_y.append(train_y[i])
        
        
        aug= transform(image= train_x[i], mask= train_y[i])
#         print(aug)
#         print(type(aug))
        new_x.append(aug['image'])
        new_y.append(aug['mask'])
        
        
    return np.array(new_x).astype(np.float64), np.array(new_y).astype(np.float64)

In [ ]:
def dice_coef(y_true, y_pred, smooth=1.0):
    ''' Dice Coefficient 
    Args:
        y_true (np.array): Ground Truth Heatmap (Label)
        y_pred (np.array): Prediction Heatmap
    '''
    class_num = 1
    for i in range(class_num):
        y_true_f = K.flatten(y_true[:,:,:,i])
        y_pred_f = K.flatten(y_pred[:,:,:,i])
        intersection = K.sum(y_true_f * y_pred_f)
        loss = ((2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth))
        if i == 0:
            total_loss = loss
        else:
            total_loss = total_loss + loss
            total_loss = total_loss / class_num
    return total_loss
    
def dice_coef_loss(y_true, y_pred):
    ''' Dice Coefficient Loss
    Args:
        y_true (np.array): Ground Truth Heatmap (Label)
        y_pred (np.array): Prediction Heatmap
    '''
    return 1-dice_coef(y_true, y_pred)
    

In [ ]:
def u_net():
#     classes means unet output channel numbers
    model = sm.Unet('efficientnetb0', classes = 1, input_shape=(None,None,3), encoder_weights='imagenet', activation='sigmoid')
    
#     tensorflow.kerasiioptimizers.Adadelta(learning_rate=0.01)
    
    model.compile(
    'sgd',
    loss= sm.losses.bce_jaccard_loss,
    metrics=[dice_coef],
    )
    
    return model

In [ ]:
# def solve_cudnn_error():
#     gpus = tf.config.experimental.list_physical_devices('GPU')
#     if gpus:
#         try:
#             # Currently, memory growth needs to be the same across GPUs
#             for gpu in gpus:
#                 tf.config.experimental.set_memory_growth(gpu, True)
#             logical_gpus = tf.config.experimental.list_logical_devices('GPU')
#             print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
#         except RuntimeError as e:
#             # Memory growth must be set before GPUs have been initialized
#             print(e)

# Train


In [ ]:
exp_code = 27
epoch = 50
best_iou = 0 

train_iou = []
train_loss =[]
val_iou = []
val_loss = []

model = u_net()
img_path = 'Train_Dev/crop_training_256'
label_path = 'Train_Dev/crop_trainlabel_image_256'


# read data and normalization
img_all, label_all, img_name_all = read_data(img_path,label_path)
img_all = minmax_normal(img_all)
label_all = minmax_normal(label_all)

print('data_shape = ' + str(img_all.shape))

#data augmentation
vali_percentage= 0.15
# #     vali_x, vali_y= img_all[-vali_count:], label_all[-vali_count:]
# # train_x, train_y= img_all[:], label_all[:]
train_x_name, vali_x_name = train_test_split(img_name_all, test_size = vali_percentage, random_state = 42)
train_x, vali_x = train_test_split(img_all, test_size = vali_percentage, random_state = 42)
train_y, vali_y = train_test_split(label_all, test_size = vali_percentage, random_state = 42)

del img_all,label_all,img_name_all
gc.collect()


for i in range(epoch):
    ep = i+1
    print('epoch:'+ str(ep))
    
    new_x, new_y = data_aug(train_x, train_y)
    
# #     # checkpoint
# #     filepath = "weight/weights_exp"+ str('%02d'%exp_code) +"-{epoch:02d}.h5"
# #     checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor = 'val_dice_coef', verbose=1, save_best_only = False,save_weights_only = False)
# #     callbacks_list = [checkpoint]

    train_history = model.fit(new_x, new_y, validation_data=(vali_x, vali_y), epochs = 1, batch_size= 5, verbose= 2)
#     train_history = model.fit(train_x, train_y, epochs = 1, batch_size= 9, verbose= 2)

    val_iou.append( train_history.history['val_dice_coef'][-1] )
    train_iou.append( train_history.history['dice_coef'][-1] )
    val_loss.append( train_history.history['val_loss'][-1] )
    train_loss.append( train_history.history['loss'][-1] )

    print('epoch: ', ep, ', ', 'val_IOU= ', val_iou[-1])
#     print('epoch: ', ep, ', ', 'train_IOU= ', train_iou[-1])

    print('val_IOU= ', val_iou[-1]) 
    print('train_IOU= ', train_iou[-1])
    print('val_LOSS= ', val_loss[-1]) 
    print('train_LOSS= ', train_loss[-1]) 
    print()

    if val_iou[-1] > best_iou:
        model.save('weight/best_model_exp' + str('%02d'%exp_code) + 'epoch_' + str('%02d'%ep) + '.h5')
        best_iou = val_iou[-1]
        print('model save at epoch: '+ str(ep) +', val_dice_coef: '+str(val_iou[-1]))
        
#     if train_iou[-1] > best_iou:
#         model.save('weight/best_model_exp' + str('%02d'%exp_code) + 'epoch_' + str('%02d'%ep) + '.h5')
#         best_iou = train_iou[-1]
#         print('model save at epoch: '+ str(ep) +', val_dice_coef: '+str(train_iou[-1]))
        
    del train_history,new_x,new_y
    gc.collect()
#     del new_x,new_y
#     gc.collect()

In [ ]:
del train_history
gc.collect()

# Output record file

In [ ]:
filename = 'exp_result/csv/exp_' + str('%02d'%exp_code) +'_history.csv'
exp_result = pd.DataFrame(list(zip(train_iou, val_iou, train_loss, val_loss)),
               columns =['train_iou', 'val_iou','train_loss','val_loss'])
exp_result.to_csv(filename)

# Visualization

In [ ]:
# plot loss
filename = 'exp_result/figure/exp_' + str('%02d'%exp_code) +'_loss.jpeg'
plt.figure(figsize=(15,10))
x_ticks = np.linspace(1,epoch,epoch)
# y_ticks = np.linspace(0,1,11)
plt.plot(x_ticks,val_loss,color='r',label='val_loss')
plt.plot(x_ticks,train_loss,color='b',label='train_loss')
plt.title('Exp_' + str('%02d'%exp_code) +' loss', fontsize=20)
plt.xlabel('Epoch',fontsize=15)
plt.ylabel('loss',fontsize=15)

plt.xlim((1,epoch))
# plt.ylim((0,1))

plt.xticks(x_ticks)
# plt.yticks(y_ticks)
plt.grid()
plt.legend(prop={'size': 15})
plt.savefig(filename)

In [ ]:
# plot Dice
filename = 'exp_result/figure/exp_' + str('%02d'%exp_code) +'_dice.jpeg'
plt.figure(figsize=(15,10))
x_ticks = np.linspace(1,epoch,epoch)
y_ticks = np.linspace(0,1,11)
plt.plot(x_ticks,val_iou,color='r',label='val_dice')
plt.plot(x_ticks,train_iou,color='b',label='train_dice')
plt.title('Exp_' + str('%02d'%exp_code) +' dice_coefficient',fontsize=20)
plt.xlabel('Epoch',fontsize=15)
plt.ylabel('dice_coefficient',fontsize=15)

plt.xlim((1,epoch))
plt.ylim((0,1))

plt.xticks(x_ticks)
plt.yticks(y_ticks)
plt.grid()
plt.legend(prop={'size': 15})
plt.savefig(filename)

# load model

In [ ]:
class FixedDropout(tf.keras.layers.Dropout):
    def _get_noise_shape(self, inputs):
        if self.noise_shape is None:
            return self.noise_shape
        symbolic_shape = K.shape(inputs)
        noise_shape = [symbolic_shape[axis] if shape is None else shape for axis, shape in enumerate(self.noise_shape)]
        return tuple(noise_shape)
    
model2 = tf.keras.models.load_model('weight/best_model_exp' + str('%02d'%exp_code) +'epoch_19'+ '.h5', compile=True, 
                                    custom_objects={'swish':tf.compat.v2.nn.swish,
                                                  'binary_crossentropy_plus_jaccard_loss':sm.losses.bce_jaccard_loss,
                                                  'dice_coef_loss':dice_coef_loss,
                                                  'dice_coef': dice_coef,
                                                  'FixedDropout':FixedDropout})

# Predict

In [ ]:
from skimage.measure import label, regionprops
from skimage.filters import threshold_otsu
import matplotlib.patches as mpatches
from skimage.morphology import square, dilation, erosion
import time

def transform_center(img, ans):
    
    ans= ans*255

    ans= ans.reshape(ans.shape[0],ans.shape[1])
    try:
        thresh = threshold_otsu(ans)
    except:
        return ''
    bw = dilation(ans > thresh, square(11)) #11
    bw = erosion(bw > thresh, square(11))
    label_image = label(bw, connectivity = 2)
    plt.imshow(bw)
    plt.show()

    """
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.imshow(img.reshape(img.shape[0],img.shape[1],3), cmap= 'gray')
    """
    
    # 紀錄所有區域之面積
    area=[]
    for region in regionprops(label_image):
        minr, minc, maxr, maxc = region.bbox
        x_width= maxc - minc
        y_width= maxr - minr
        area.append(x_width*y_width)
    # 排序大小
    index_large= sorted(range(len(area)), key = lambda k : area[k])
    index_large.reverse()
    region_prob= []
    for i in range(len(regionprops(label_image))):
        region_prob.append( regionprops(label_image)[index_large[i]] )
    
    #region_prob= region_prob[:int(len(region_prob)*0.9)]
        
        
    all_center= []
    print('region_bbox: ', len(region_prob))
    for region in region_prob:
        minr, minc, maxr, maxc = region.bbox
        x_width= maxc - minc
        y_width= maxr - minr
        minr= minr-y_width/2
        minc= minc-x_width/2
        x_width= x_width*2
        y_width= y_width*2
        
        
        if (maxc - minc)>0 and (maxr - minr)>0:
            all_center.append([int(minc+x_width/2), int(minr+y_width/2)])
            """
            rect = mpatches.Rectangle((minc, minr), x_width, y_width,
                                        fill=False, edgecolor='red', linewidth=2)
            ax.add_patch(rect)
            """
            
            
    #plt.show()
    
    return all_center

In [ ]:
# load data
def read_data_predict(img_path):
    img_name = os.listdir(img_path)
    img_all = []
    
    for img_tmp in img_name:
        img = cv2.imread(img_path + '/' + img_tmp)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
#         #resize
#         resize_number = 256
#         img = cv2.resize(img,(resize_number,resize_number))
        
        img_all.append(img)
    return np.array(img_all)

# Output

In [ ]:
import math
#load data
img_path = 'test/crop_test_private_256_hist/'
test_img = read_data_predict(img_path)
test_img = minmax_normal(test_img)

# predict
predict_mask = model2.predict(test_img,batch_size = 16)

# resize
resize_number = 256 
threshold = 0.5
print('orginal size:',predict_mask.shape)
process_mask = [cv2.resize(predict_mask[i],(resize_number,resize_number)) for i in range(len(predict_mask))]
process_mask = np.array(process_mask)
process_mask_t = process_mask > threshold
process_mask_t = process_mask_t.astype(float)
print('after preocss size:',process_mask_t.shape)

# output
# 1.先找圖片名字
img_name_path = 'test_private/'
img_list = os.listdir(img_name_path)
ful_img_name = [img_list[i].split('.')[0]for i in range(len(img_list))]

# 2.讀入所有圖片
spt_img_name = os.listdir(img_path)
cnt=0

# 3.針對圖片做輸出
for img_name in ful_img_name:
    print(img_name)
    if(len(img_name)<10):
        o_x = 3000
        o_y = 2000
        small_x = math.ceil(o_x / resize_number)
        small_y = math.ceil(o_y / resize_number)
        small_img_number = small_x * small_y
        full_img = Image.new('L',(small_x * resize_number , small_y * resize_number))
    else:
        o_x = 2304
        o_y = 1728
        small_x = math.ceil(o_x / resize_number)
        small_y = math.ceil(o_y / resize_number)
        small_img_number = small_x * small_y
        full_img = Image.new('L',(small_x * resize_number , small_y * resize_number))
    cnt = 0
    for y in range(small_y):
        for x in range(small_x):
            cnt = cnt+1
            img_code = img_name  + '_' + str(cnt)  +'.JPG'
            idx = spt_img_name.index(img_code)
#             print(spt_img_name[idx])
#             print(y,x)
            small_img = Image.fromarray(process_mask_t[idx])
            full_img.paste(small_img,(x * resize_number,y *  resize_number))
    if(len(img_name)<10): 
        ans = np.array(full_img)
    else:
        ans = np.array(full_img) 
        ans = ans[0:1728,0:2304]
#     ans = np.array(full_img)
    ans_img = cv2.imread(img_name_path + img_name +'.JPG')
    bbox = transform_center(ans_img,ans)
    radius = 5
    for box in bbox:
        cv2.rectangle(ans_img,
                      (box[0]-radius, box[1]-radius),
                      (box[0]+radius, box[1]+radius),
                      (255, 255, 255),
                      -1)
    for x in range(0,o_x,resize_number):
        for y in range(0,o_y,resize_number):
            cv2.rectangle(ans_img,
                      (x-radius, y-radius),
                      (x+radius, y+radius),
                      (0, 0, 255),
                      2)      
#     save image
    cv2.imwrite('predict_test_private/exp_' + str('%02d'%exp_code) +'/img/'+ img_name +'.JPG',ans_img) #save labelimage
#     save csv file
    filename_1 = 'predict_test_private/exp_' + str('%02d'%exp_code) +'/csv/' + img_name +'.csv'
    filename_2 = 'predict_upload/exp_' + str('%02d'%exp_code) +'/' + img_name +'.csv'
    exp_result = pd.DataFrame(bbox , columns =['X','Y'])
    exp_result.to_csv(filename_1,header = None, index = False)
    exp_result.to_csv(filename_2,header = None, index = False)

# compare label and result

In [ ]:
all_label_name = os.listdir('Train_Dev/train_labels')

radius = 1
for label_name in all_label_name[0:1]:
    # read image
    img = cv2.imread('Train_Dev/training/' + label_name.split('.')[0] + '.JPG')
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    label = pd.read_csv('Train_Dev/train_labels/' + label_name, header= None)# read label
    label_img = np.zeros(img.shape, dtype = 'uint8')#make label image
    
    bbox = label.values
    for box in bbox:
        cv2.rectangle(label_img,
                      (box[0]-radius, box[1]-radius),
                      (box[0]+radius, box[1]+radius),
                      (255, 255, 255),
                      -1)
    cv2.imwrite('Train_Dev/train_labels_image/'+ label_name.split('.')[0] +'.JPG',label_img) #save labelimage
    
    plt.imshow(img)
    plt.show()
    plt.imshow(label_img)
    plt.show()

In [ ]:
import math
resize_number=256

o_x=3000
o_y=2000
bounds = []
for x in range(0,o_x,resize_number):
    for y in range(0,o_y,resize_number):
        bound = [x,y]
        bounds.append(bound)
# for box in bounds:
#         cv2.rectangle(ans_img,
#                       (box[0]-rr, box[1]-rr),
#                       (box[0]+rr, box[1]+rr),
#                       (0, 0, 255),
#                       2)

In [ ]:
print(bounds)

In [ ]:
for i in range(0,3000,256):
    print(i)

In [ ]:
resize_number*()